In [125]:
"""Reading of Excel File 
and importing packages"""

import numpy as np
import scipy.stats as ss
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_excel("/Users/rohitjain/Downloads/NFLX.xlsx",header=0,index_col=None) #loading the csv file


In [126]:
from sklearn import neighbors
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler


In [142]:
"""Feature Engineering"""

df['OC']=df['Open']-df['Close']
df['AvgOC']=(df['Open']+df['Close'])/2
df['HL']=df['High']-df['Low']
df['AvgHL']=(df['High']+df['Low'])/2
df['LV']=np.log(df['Volume'])
"Selected Feature"
X=df[['Close','HL','OC','LV']]

"""Traget Variable"""
Y1=df['Open'].as_matrix()
Y1=Y1[0:670]

"Target variable Y as next day Opening value"
Y=df['Open'].shift(-1).as_matrix()
T=670

X=X[0:670]
X=np.column_stack([np.ones((T,1)),X])

Y=Y[0:670]

"""Random Split into Train and test with pareto principle of 80:20"""
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.00, random_state=42)


/Users/rohitjain/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
/Users/rohitjain/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


In [143]:
Y1

array([111.55      ,  57.49132101,  18.65114421,  30.66781942,
        28.75132158,  28.30470011,  44.71582913,  28.0694823 ,
        21.54555723,  25.40526429,  35.91908992,  27.80445566,
        38.78318534,  28.82531484,  41.11919013,  31.46784956,
        34.16114439,  30.34056581,  24.70010497,  38.64051515,
        32.69098892,  38.19122279,  44.28730876,  31.08829914,
        26.05607303,  35.25137218,  27.11034761,  34.76994157,
        35.83078511,  33.01775364,  25.64817526,  20.05536373,
        28.7647371 ,  37.85717794,  38.07936219,  25.920275  ,
        33.36132577,  21.99572536,  33.61352238,  45.31984554,
        32.77469115,  31.37925668,  34.38996454,  38.25808885,
        38.1644029 ,  34.9876608 ,  29.27256876,  28.08731901,
        18.87759321,  45.15777023,  37.96911325,  38.29038418,
        20.67962489,  31.04848501,  32.48015739,  29.85690188,
        30.42149731,  22.19023678,  26.22746139,  27.02087065,
        30.8643704 ,  22.5961039 ,  41.48841879,  37.39

In [144]:
from scipy.optimize import minimize


In [145]:
def linear(Xt,Yt):
    invXX = np.linalg.inv(Xt.transpose()@Xt)
    beta_hat = invXX@Xt.transpose()@Yt
    y_hat = Xt@beta_hat
    return y_hat


In [146]:
def GARCH(Y1):
    mu = param0[0]
    omega = param0[1]
    alpha = param0[2]
    beta = param0[3]
    T = Y1.shape[0]
    GARCH_Dens = np.zeros(T)
    sigma2 = np.zeros(T)
    F = np.zeros(T)
    v = np.zeros(T)
    sigma2[0] = omega/(1-alpha)
    
    for t in range(1,T):
        sigma2[t] = omega+alpha*((Y1[t-1]-mu)**2)+beta*(sigma2[t-1])
        F[t] = Y1[t] - mu-np.sqrt(sigma2[t])*np.random.normal(0,1,1)
        v[t] = sigma2[t];
        GARCH_Dens[t] = (1/2)*np.log(2*np.pi)+(1/2)*np.log(v[t])+\
                     (1/2)*(F[t]/v[t]);
        Likelihood = np.sum(GARCH_Dens[1:-1])
    return Likelihood

In [147]:
def GARCH_PROD(params,T,Y1):
    mu = param0[0]
    omega = param0[1]
    alpha = param0[2]
    beta = param0[3]
    #Y = np.zeros(T)
    sigma2 = np.zeros(T)
    sigma2[0]= omega/(1-alpha)
    for t in range(1,T):
        sigma2[t] = omega+alpha*((Y1[t-1]-mu)**2)+beta*(sigma2[t-1])
        Y1[t] = mu+np.sqrt(sigma2[t])*np.random.normal(0,1,1)
    return Y1
    

In [148]:
"""Not Used Yet Try to implement"""
def GARCH_t(Y):
    mu = param0[0]
    omega = np.exp(param0[1])
    alpha = 1/(1+np.exp(-param0[2]))
    beta = param0[3]
    nv = param0[4]
    
    T = Y.shape[0]
    GARCH_t_dens = np.zeros(T)
    sigma2 = np.zeros(T)
    F = np.zeros(T)
    V = np.zeros(T)
    sigma2[0] = omega/(1-alpha)
    
    for t in range(1,T):
        sigma2[t] = omega+alpha*((Y[t-1]-mu)**2)+beta*(sigma2[t-1])
        F[t] = Y[t] - mu-np.sqrt(sigma2[t])*np.random.standard_t(nv,1)
        V[t] = sigma2[t];
        GARCH_t_dens[t] = np.log(ss.gamma((nv+1)/2))-np.log(np.sqrt(nv*np.pi))-\
                          np.log(ss.gamma(nv/2))-((nv+1)/2)*np.log(1+((F[t]**2)/V[t])/nv)
        Likelihood = np.sum(GARCH_t_dens[1:-1])
    return Likelihood

In [149]:
def GARCH_PROD_t(params, Y0, T):
    mu = params[0]
    omega = np.exp(params[1])
    alpha = 1/(1+np.exp(-params[2]))
    beta = params[3]
    nv = params[4]
    #Y = np.zeros(T)
    Y = (df['Open ']+df['Close'])/2
    #Y = (df['Volume'])
    sigma2 = np.zeros(T)
    Y[0] = Y0
    sigma2[0]= omega/(1-alpha)
    for t in range(1,T):
        sigma2[t] = omega+alpha*((Y[t-1]-mu)**2)+beta*(sigma2[t-1]);
        Y[t] = mu+np.sqrt(sigma2[t])*np.random.normal(nv,1)
    return Y

In [150]:
def Kalman_Filter(YK):

    S=YK.shape[0]
    z = 1
    Z = 1
    T = 0.5
    H = np.var(YK);
    Q = 0.5*np.var(YK)
    #Kalman Filter Starts
    u_predict = np.zeros(S)
    u_update = np.zeros(S)
    p_predict = np.zeros(S)
    p_update = np.zeros(S)
    v = np.zeros(S)
    F = np.zeros(S)
    KF_Dens = np.zeros(S)

    for s in range(1,S):
        if s ==1:
            p_update[s] =1000
            p_predict[s] = T*p_update[1]*np.transpose(T)+Q
        else:
            F[s] = z*p_predict[s-1]*np.transpose(z)+H
            v[s] = YK[s-1]-z*u_predict[s-1]

            u_update[s] =T*u_update[s-1]+p_predict[s-1]*np.transpose(Z)*(1/F[s])*v[s]
            u_predict[s] = T*u_update[s]

            p_update[s] = p_predict[s-1]-p_predict[s-1]*np.transpose(Z)*(1/F[s])*Z*p_predict[s-1]
            p_predict[s] = T*p_update[s]*np.transpose(T)+Q
            KF_Dens[s]=(1/2)*np.log(2*np.pi)+(1/2)*np.log(abs(F[s]))+(1/2)*np.transpose(v[s])*(1/F[s])
            Likelihood=sum(KF_Dens[1:-1])
    return Likelihood

In [151]:
def Kalman_Smoother(params,YKS):
    S=YKS.shape[0]
    Z = params[0]
    T = params[1]
    H = params[2]
    Q = params[3]
    #Kalman Filter Starts
    u_predict = np.zeros(S)
    u_update = np.zeros(S)
    p_predict = np.zeros(S)
    p_update = np.zeros(S)
    v = np.zeros(S)
    F = np.zeros(S)
    KF_Dens = np.zeros(S)

    for s in range(1,S):
        if s ==1:
            p_update[s] =1000
            p_predict[s] = T*p_update[1]*np.transpose(T)+Q
        else:
            F[s] = Z*p_predict[s-1]*np.transpose(Z)+H
            v[s] = YKS[s-1]-Z*u_predict[s-1]
            
            u_update[s] =T*u_update[s-1]+p_predict[s-1]*np.transpose(Z)*(1/F[s])*v[s]
            u_predict[s] = T*u_update[s]

            p_update[s] = p_predict[s-1]-p_predict[s-1]*np.transpose(Z)*(1/F[s])*Z*p_predict[s-1]
            p_predict[s] = T*p_update[s]*np.transpose(T)+Q
        
    u_smooth = np.zeros(S)
    p_smooth = np.zeros(S)
    u_smooth[S-1] = u_update[S-1]
    p_smooth[S-1] = p_update[S-1]
    for t in range (S-1,0,-1):
        u_smooth[t-1] =u_update[t]+p_update[t]*np.transpose(T)/p_predict[t]*(u_smooth[t]-T*u_update[t])
        p_smooth[t-1] = p_update[t]+p_update[t]*np.transpose(T)/p_predict[t]*(p_smooth[t]-p_predict[t])/p_predict[t]
    return u_smooth

In [152]:
def MovingAvg(x, y):
    preds = []
    for i in range(0,y.shape[0]):
        a = df['Open'][len(df)-301+i:].sum() + sum(preds)
        b = a/y.shape[0]
        preds.append(b)
    return preds

In [153]:
Y1Temp=Y1

In [154]:
val=MovingAvg(X,Y1)

In [155]:
df_yhatLinearTrain=pd.DataFrame()
df_yhatGarchTrain=pd.DataFrame()
df_yhatGarchTTrain=pd.DataFrame()
df_yhatKalman=pd.DataFrame()
df_yhatMA=pd.DataFrame()

df_yhatLinearTest=pd.DataFrame()
df_yhatGarchTest=pd.DataFrame()
df_yhatGarchTTest=pd.DataFrame()
Y1Temp=Y1

for x in range(0,100):
    """Linear for Train dataset"""
    df_yhatLinearTrain[x]=linear(X_train,y_train)
    
#     """Linear for Test dataset"""
#     df_yhatLinearTest[x]=linear(X_test,y_test)

    """Garch for Train dataset"""
    Y1=Y1Temp
    T = len(Y1)
    param0 = np.array([32,35,0.1,0.1])
    param_star = minimize(GARCH, param0, method ='BFGS', options ={'xtol':1e-8, 'disp':True})
    Y_GARCH = GARCH_PROD(param_star.x,T,Y1)
    df_yhatGarchTrain[x]=Y_GARCH
    
    Y1=Y1Temp
    param0 = np.array([1.3,0.3,8,9])
    param_star = minimize(Kalman_Filter, param0, method = 'BFGS', options ={'xtol':1e-8,'disp': True})
    Y_update = Kalman_Smoother(param_star.x,Y1)
    df_yhatKalman[x]=Y_update
    
    Y1=Y1Temp
    df_yhatMA[x]=MovingAvg(X,Y1)
#     """Garch for Test dataset"""
#     T = len(y_test)
#     mu = 35
#     sig =5
#     param0 = np.array([32,35,0.2,0.5])
#     param_star = minimize(GARCH, param0, method ='BFGS', options ={'xtol':1e-8, 'disp':True})
#     Y_GARCH = GARCH_PROD(param_star.x,T,y_test)
#     df_yhatGarchTest[x]=Y_GARCH


/Users/rohitjain/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: OptimizeWarning: Unknown solver options: xtol
/Users/rohitjain/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: OptimizeWarning: Unknown solver options: xtol


         Current function value: 4.964335
         Iterations: 3
         Function evaluations: 132
         Gradient evaluations: 20
         Current function value: -23038.237954
         Iterations: 51
         Function evaluations: 420
         Gradient evaluations: 68


/Users/rohitjain/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: OptimizeWarning: Unknown solver options: xtol
/Users/rohitjain/anaconda3/lib/python3.7/site-packages/scipy/optimize/optimize.py:1013: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (numpy.dot(yk, sk))
/Users/rohitjain/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: OptimizeWarning: Unknown solver options: xtol


Divide-by-zero encountered: rhok assumed large
         Current function value: 4.899201
         Iterations: 3
         Function evaluations: 249
         Gradient evaluations: 39
         Current function value: -23038.237954
         Iterations: 51
         Function evaluations: 420
         Gradient evaluations: 68


/Users/rohitjain/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: OptimizeWarning: Unknown solver options: xtol
/Users/rohitjain/anaconda3/lib/python3.7/site-packages/scipy/optimize/optimize.py:1013: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (numpy.dot(yk, sk))
/Users/rohitjain/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: OptimizeWarning: Unknown solver options: xtol


...

         Current function value: 4.992882
         Iterations: 1
         Function evaluations: 156
         Gradient evaluations: 24
         Current function value: -23038.237954
         Iterations: 51
         Function evaluations: 420
         Gradient evaluations: 68


/Users/rohitjain/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: OptimizeWarning: Unknown solver options: xtol
/Users/rohitjain/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: OptimizeWarning: Unknown solver options: xtol


         Current function value: 4.875500
         Iterations: 1
         Function evaluations: 114
         Gradient evaluations: 17
         Current function value: -23038.237954
         Iterations: 51
         Function evaluations: 420
         Gradient evaluations: 68


/Users/rohitjain/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: OptimizeWarning: Unknown solver options: xtol
/Users/rohitjain/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: OptimizeWarning: Unknown solver options: xtol


         Current function value: 5.052162
         Iterations: 1
         Function evaluations: 114
         Gradient evaluations: 17
         Current function value: -23038.237954
         Iterations: 51
         Function evaluations: 420
         Gradient evaluations: 68


In [156]:
resdata={}
y_hatTrainLinear=df_yhatLinearTrain.mean(axis=1)
reslinT=y_train-y_hatTrainLinear

y_hatTrainGarch=df_yhatGarchTrain.mean(axis=1)
resG=Y1Temp-y_hatTrainGarch

y_hatKalman=df_yhatKalman.mean(axis=1)
resK=Y1Temp-y_hatKalman


y_hatMA=df_yhatMA.mean(axis=1)
resMA=Y1Temp-y_hatMA

resdata={'Linear':reslinT,'Garch':resG,'Kalman':resK,'Moving Average':resMA}


In [157]:
def randomForest(datadict):
    data={'Algo':'','RMSE':None}
    for key,val in datadict.items():
        rmse=np.sqrt(((val) ** 2).mean())
        if(data['RMSE']==None):
            data['Algo']=key
            data['RMSE']=rmse
        elif data['RMSE']>rmse:
            data['Algo']=key
            data['RMSE']=rmse
    return data

In [158]:

rmse=np.sqrt(((reslinT) ** 2).mean())
rmse

14.50520937197714

In [159]:
ranndomforestoutput=randomForest(resdata)

In [161]:
ranndomforestoutput

{'Algo': 'Garch', 'RMSE': 6.7547459303886805}

# RMSE for Garch is 6.73 and output received from Random Forest is GARCH